 # Table of Contents
<div class="toc" style="margin-top: 1em;"><ul class="toc-item" id="toc-level0"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#VGG16-model" data-toc-modified-id="VGG16-model-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>VGG16 model</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Embedding-of-COCO-images" data-toc-modified-id="Embedding-of-COCO-images-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Embedding of COCO images</a></span></li></ul></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#LSTM-+-VGG" data-toc-modified-id="LSTM-+-VGG-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>LSTM + VGG</a></span><ul class="toc-item"><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-text-data" data-toc-modified-id="Prepare-text-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Prepare text data</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Prepare-image-data" data-toc-modified-id="Prepare-image-data-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Prepare image data</a></span></li><li><span><a href="http://localhost:8888/notebooks/Documents/git/vqa/demos/lstm_vgg_demo.ipynb#Model-definition" data-toc-modified-id="Model-definition-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Model definition</a></span></li></ul></li></ul></div>

In [1]:
# Add the path of the main directory
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

In [2]:
import matplotlib.pyplot as plt
import skimage.io as io
import numpy as np
import random
import keras

from keras.layers import Dense, Dropout, LSTM, multiply
from keras.layers.embeddings import Embedding
from keras.utils import plot_model
from keras.layers import Input
from keras.models import Model



Using Theano backend.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


In [3]:
from vqa_api.PythonHelperTools.vqaTools.vqa import VQA

In [4]:
%load_ext autoreload

%autoreload 2

# VGG16 model

In [18]:
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from tools import img_dir, img_file

In [5]:
# VGG16 model
vgg_model = keras.applications.vgg16.VGG16(include_top=True, weights='imagenet', 
                               input_tensor=None, input_shape=None, 
                               pooling=None, classes=1000)

In [6]:
# Model layers 
layers = vgg_model.layers
for i in range(len(layers)):
    print "--> Layer {}".format(i+1), "\n"
    print layers[i].get_config(), "\n"
    

--> Layer 1 

{'dtype': 'float32', 'batch_input_shape': (None, 224, 224, 3), 'name': 'input_1', 'sparse': False} 

--> Layer 2 

{'padding': 'same', 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'block1_conv1', 'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'activation': 'relu', 'trainable': True, 'data_format': 'channels_last', 'kernel_constraint': None, 'strides': (1, 1), 'dilation_rate': (1, 1), 'kernel_regularizer': None, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'use_bias': True, 'activity_regularizer': None, 'kernel_size': (3, 3)} 

--> Layer 3 

{'padding': 'same', 'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'block1_conv2', 'bias_regularizer': None, 'filters': 64, 'bias_constraint': None, 'activation': 'relu', 'trainable': True, 'data_

The fc7 layer is the layer 22, which name is fc2. 

In [7]:
# fc7 layer
fc7_layer = layers[21]
print fc7_layer.get_config()
fc7_layer_name = fc7_layer.get_config()["name"]
print fc7_layer_name

{'kernel_initializer': {'class_name': 'VarianceScaling', 'config': {'distribution': 'uniform', 'scale': 1.0, 'seed': None, 'mode': 'fan_avg'}}, 'name': 'fc2', 'kernel_constraint': None, 'bias_regularizer': None, 'bias_constraint': None, 'activation': 'relu', 'trainable': True, 'kernel_regularizer': None, 'bias_initializer': {'class_name': 'Zeros', 'config': {}}, 'units': 4096, 'use_bias': True, 'activity_regularizer': None}
fc2


In [8]:
# Input shape and output shape of the fc7 layer
print "Input shape:", fc7_layer.input_shape
print "Ouput shape:", fc7_layer.output_shape

Input shape: (None, 4096)
Ouput shape: (None, 4096)


In [9]:
# Weights of the fc7 layer
# Weights shape
print "Weights shape:", fc7_layer.get_weights()[0].shape
# Bias shape
print "Bias shape:", fc7_layer.get_weights()[1].shape

Weights shape: (4096L, 4096L)
Bias shape: (4096L,)


We can also get a Layer by its name:

In [10]:
# fc7 layer
fc7_layer = vgg_model.get_layer(fc7_layer_name)
fc7_layer.get_config()

{'activation': 'relu',
 'activity_regularizer': None,
 'bias_constraint': None,
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'bias_regularizer': None,
 'kernel_constraint': None,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'distribution': 'uniform',
   'mode': 'fan_avg',
   'scale': 1.0,
   'seed': None}},
 'kernel_regularizer': None,
 'name': 'fc2',
 'trainable': True,
 'units': 4096,
 'use_bias': True}

## Embedding of COCO images 

In [16]:
dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"
dataType = "mscoco"

In [20]:
image_ids = [(9, "train2014"), (25, "train2014")]
# List to store the arrays for images
imgs = []
for image_id in image_ids:
    # Resize the images as VGG inputs
    img = image.load_img(os.path.join(img_dir(dataDir, dataType, 
        image_id[1]), img_file(image_id[1], image_id[0])), target_size=(224, 224)) 
    img = image.img_to_array(img)
    imgs.append(img)
imgs = np.stack(imgs)
# Preprocess the images corresponding to VGG
imgs = preprocess_input(imgs)   

In [21]:
imgs.shape

(2L, 224L, 224L, 3L)

In [24]:
# "Submodel" of VGG until the fc7 layer
vgg_model_fc7 = Model(inputs=vgg_model.input, outputs=vgg_model.get_layer("fc2").output)

In [26]:
# Images embedding using the fc7 layer of VGG16
vgg_model_fc7.predict(imgs).shape

(2L, 4096L)

# LSTM + VGG

In [6]:
from lstm_vgg import LSTMVGG

## Prepare text data 

In [7]:
dataDir = r"C:\Users\Nasser Benab\Documents\git\data\vqa"

In [8]:
text_vision = LSTMVGG(dataDir, n_train=300, n_test=300)

C:\Users\Nasser Benab\Documents\git\data\vqa\Annotations\mscoco_train2014_annotations.json
--> train2014
loading VQA annotations and questions into memory...
0:00:13.028000
creating index...
index created!
loading VQA annotations and questions into memory...
0:00:06.845000
creating index...
index created!


In [9]:
# Tokenize questions using the vocabulary of the training questions
train_sequences, test_sequences = text_vision.tokenize_questions()

In [10]:
# Number of questions in the train and test set
len(train_sequences), len(test_sequences)

(300, 300)

In [ ]:
# Maximum length for the input questions
input_length = 22

# Vocabulary size from the training set
V = 13666
input_dim = V + 1

# Dimension of the vectors (from the paper)
embedding_dim = 300

# The output from the embedding layer is equal to input_length x embedding_dim

## Prepare image data

In [13]:
train_images, test_images = text_vision.process_images()

100%|██████████| 100/100 [00:01<00:00, 93.98it/s]


In [14]:
train_images.shape

(300L, 224L, 224L, 3L)

In [20]:
text_vision.vqa_train2014.dataset["annotations"][0]

{u'answer_type': u'other',
 u'answers': [{u'answer': u'oval',
   u'answer_confidence': u'yes',
   u'answer_id': 1},
  {u'answer': u'semi circle', u'answer_confidence': u'yes', u'answer_id': 2},
  {u'answer': u'curved', u'answer_confidence': u'yes', u'answer_id': 3},
  {u'answer': u'curved', u'answer_confidence': u'yes', u'answer_id': 4},
  {u'answer': u'double curve', u'answer_confidence': u'yes', u'answer_id': 5},
  {u'answer': u'banana', u'answer_confidence': u'maybe', u'answer_id': 6},
  {u'answer': u'curved', u'answer_confidence': u'yes', u'answer_id': 7},
  {u'answer': u'wavy', u'answer_confidence': u'yes', u'answer_id': 8},
  {u'answer': u'twisting', u'answer_confidence': u'no', u'answer_id': 9},
  {u'answer': u'curved', u'answer_confidence': u'maybe', u'answer_id': 10}],
 u'image_id': 487025,
 u'multiple_choice_answer': u'curved',
 u'question_id': 4870250,
 u'question_type': u'what'}

## Model definition

In [ ]:
# Let's get a tensor with the output of our vision model
image_input = Input(shape=(224, 224, 3))
# Freeze the vision model weights
vgg_model_fc7.trainable = False
fc7 = vgg_model_fc7(image_input)

# Turn the 4096 embedding from the fc7 layer to a 1024 embedding in order to 
# match the questions embedding
encoded_image = Dense(1024, activation="tanh")(fc7)

# Langage model
question_input = Input(shape=(100,), dtype='int32')
embedded_question = Embedding(input_dim=input_dim, output_dim=embedding_dim, 
                              input_length=input_length)(question_input)
encoded_question = LSTM(1024)(embedded_question)

# Point-wise multiplication of the outputs from the vision model and the 
# langage model
merge = multiply([encoded_question, encoded_image])

# Hidden layers with 0.5 dropouts
merge = Dropout(0.5)(merge)
hidden1 = Dense(1000, activation="tanh")(merge)
hidden1 = Dropout(0.5)(hidden1)
hidden2 = Dense(1000, activation="tanh")(hidden1)

# Final softmax layer
output = Dense(1000, activation='softmax')(hidden2)

# This is our final model:
vqa_model = Model(inputs=[image_input, question_input], outputs=output)

# Summarize layers
print(vqa_model.summary())
# Plot graph
plot_model(model, to_file="vision_model.png")